### Init Spark Session and Spark Dataframes
Here we start our spark session and we also connect to our postgres DB which is running in another container.
From the DB we read out the tables **litter_bin_geoposition** and **bin_state_notification** and save them as a Spark Dataframe.

In [1]:
from pyspark.sql import SparkSession
import uuid
import random
import datetime
import numpy as np

spark = SparkSession.builder \
    .appName("SmartLitter") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://db:5432/litter_db"
connection_properties = {
    "user": "root",
    "password": "pwd123",
    "driver": "org.postgresql.Driver",
    "stringtype": "unspecified"
}

litter_bin_geoposition_df = spark.read.jdbc(
    url=jdbc_url,
    table="litter_bin_geoposition",
    properties=connection_properties
)

bin_state_notification_df = spark.read.jdbc(
    url=jdbc_url,
    table="bin_state_notification",
    properties=connection_properties
)
litter_bin_geoposition_df.head(2)
bin_state_notification_df.head(2)

[Row(state_uuid='e0271740-0585-493a-bbd2-240405edfa6a', litter_bin_uuid='8431c864-7ea2-4a9d-9e96-855113736445', point_in_time=datetime.datetime(2023, 11, 15, 20, 34), notification_type='full', volume=240),
 Row(state_uuid='680fd2e4-b2b0-435d-88e2-409acb5c6ee7', litter_bin_uuid='8431c864-7ea2-4a9d-9e96-855113736445', point_in_time=datetime.datetime(2023, 11, 16, 19, 59), notification_type='full', volume=240)]

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 35610)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

Create **notification** table for spark df sql selects.

In [2]:
bin_state_notification_df.createOrReplaceTempView("notification")

Select the newest entries of all bins where the newest state is "full".

In [3]:
full_bins_df = spark.sql("""SELECT litter_bin_uuid, notification_type FROM (SELECT
        notification.*,
        ROW_NUMBER() OVER (PARTITION BY litter_bin_uuid ORDER BY point_in_time DESC) AS row_num
    FROM notification
) AS ranked
WHERE row_num = 1 and notification_type = 'full';""")
print(full_bins_df.count())

166


### Usage Prediction Model
Here we use the prediction model to check every bin where the newest entry says that the bin is empty.
For every bin gets predicted if it should be full by now, for this to work properly we give the amount of hours that passed since it was emptied.
We can also give an amount of hours, so if we want to run the script in the evening we give delta_hours 10 to get the prediction if the bins will be full tomorrow morning.
The result we get is a dataframe which contains every bin uuid and if the prediction is if its full or not.

In [4]:
from lib.bin import BinFullPredictor
from pyspark.sql.functions import col


predictor = BinFullPredictor()

prediction = predictor.get_all_bin_predictions(delta_hours=-20)

prediction.show()


+--------------------+-------+
|     litter_bin_uuid|is_full|
+--------------------+-------+
|01c83801-b6b1-461...|  false|
|02e3367d-58d4-449...|  false|
|03b81394-2510-443...|   true|
|04d5b3a6-5e41-460...|  false|
|04f781a6-6fa3-411...|  false|
|069b61bb-f0c9-452...|  false|
|06d073b3-7c35-4b4...|  false|
|06ec6ff5-572c-487...|   true|
|07ed5358-07bc-437...|   true|
|0834772a-159d-4e7...|  false|
|092deae5-77ad-49c...|  false|
|0954074f-1c88-4d3...|   true|
|09bd99c3-8e0f-470...|   true|
|0a36208e-b062-4b2...|  false|
|0a7ea5db-e202-4c4...|  false|
|0aa9c72b-f00a-48b...|  false|
|0b13e3e1-741e-433...|   true|
|0c5e6808-42e9-480...|  false|
|0c82b4a5-d21b-4ae...|  false|
|0cbeafbf-4e69-4b1...|  false|
+--------------------+-------+
only showing top 20 rows



Here we just convert the dataframes into list which contain the uuids of the bins.
For the bins which are empty we only the take the ones where our model predicted that they should be full.

In [5]:
predicted_full_bins_uuids = prediction.filter(col("is_full") == 'true').select("litter_bin_uuid").rdd.flatMap(lambda x: x).collect()
print(f"Number of bins which are predicted to be full: {len(predicted_full_bins_uuids)}")

full_bins_uuids = full_bins_df.select("litter_bin_uuid").rdd.flatMap(lambda x: x).collect()
print(f"Number of bins which are full: {len(full_bins_uuids)}")

full_bins_uuids = full_bins_uuids + predicted_full_bins_uuids
print(f"Number of bins which will be taken into consideration for the route planning: {len(full_bins_uuids)}")

Number of bins which are predicted to be full: 116
Number of bins which are full: 166
Number of bins which will be taken into consideration for the route planning: 282


Map the uuids of the bins to the geometric points which are stored in the geoposition table.

In [6]:
filtered_df = litter_bin_geoposition_df[litter_bin_geoposition_df['litter_bin_uuid'].isin(full_bins_uuids)]

# Extract the 'geom_point' field for the matching UUIDs
geom_points = filtered_df.select('geom_point').rdd.flatMap(lambda x: x).collect()
geom_points[:5]

['01010000200808000091ED7C2FACB943411283C08A239E3241',
 '01010000200808000008AC1C9A68C14341986E128335AB3241',
 '0101000020080800009CC420709DB7434117D9CEF7709F3241',
 '0101000020080800004A0C02EBA6B843418FC2F548739F3241',
 '010100002008080000508D972EFEB84341621058F98A9F3241']

Convert Geometric points into coordinates.
Also we add the start point of the litter route.

In [7]:
import binascii
import struct

import pandas as pd
from pyproj import Transformer


PROJECTION_FROM = "EPSG:2056"
PROJECTION_TO = "EPSG:4326"
EARTH_RADIUS = 6371.0


def extract_lat_lon(geom_point: str) -> tuple:
    x, y = struct.unpack('<dd', binascii.unhexlify(geom_point[18:]))
    transformer = Transformer.from_crs(PROJECTION_FROM, PROJECTION_TO)
    lon, lat = transformer.transform(x, y)

    return (lon, lat)


coords = [extract_lat_lon(geom_point) for geom_point in geom_points]
mainbase: tuple = (47.125835, 7.2596615)
coords = [mainbase] + coords
print(len(coords))
print(coords[:2])

283
[(47.125835, 7.2596615), (47.132004224182516, 7.246619818672296)]


### Distance Matrix as the crow flies
In this block we calculate the distance matrix based on the coordinates that we have using the haversine equation.

In [8]:
from math import sin, cos, sqrt, atan2, radians 
def haversine(lat1, lon1, lat2, lon2):
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return round(EARTH_RADIUS * c * 1000)

def create_distance_matrix(coords) -> list[list[float]]:

    n = len(coords)

    distance_matrix = []

    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(0)
            else:
                lat1, lon1 = coords[i]
                lat2, lon2 = coords[j]

                distance = haversine(radians(lat1), radians(lon1), radians(lat2), radians(lon2))
                row.append(distance)
        distance_matrix.append(row)
    
    return distance_matrix

distance_matrix: list[list[float]] = create_distance_matrix(coords=coords)

print(f"Shape: ({len(distance_matrix)}, {len(distance_matrix[0])})")

Shape: (283, 283)


### TSP Route planning algorithm
Here we use the google TSP algortihm to find the 2 shortest routes using the distance matrix.
This step can take a lot of time if we use a lot of destinations. For ~300 destinations the calculations could take up to 5 minutes.

In [9]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


demands = [1 if i != 0 else 0 for i in range(len(distance_matrix[0]))]
capacity = int((len(distance_matrix[0])-1) /2)

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["demands"] = demands
    data["vehicle_capacities"] = [capacity, capacity]
    data["num_vehicles"] = 2
    data["depot"] = 0
    return data


def print_solution(data,manager, routing, solution):
    route_indices = [[],[]]  # This will store the indices of the route
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            route_indices[vehicle_id].append(manager.IndexToNode(index))
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)})\n"
        route_indices[vehicle_id].append(manager.IndexToNode(index))
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")

    # Print the route indices
    return route_indices

# Instantiate the data problem.
data = create_data_model()


# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Transit_callback
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,      # no slack
    800000,  # vehicle maximum travel distance
    True,   # start cumul to zero
    dimension_name
)

# # Get distance dimension and set global span cost
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
route_indices = print_solution(data, manager, routing, solution)

# Make new lists with the coordinates based on the ordered indices
ordered_routes = [[coords[i] for i in vehicle] for vehicle in route_indices]

Route for vehicle 0:
 0 ->  250 ->  49 ->  280 ->  134 ->  282 ->  251 ->  275 ->  276 ->  273 ->  278 ->  274 ->  279 ->  185 ->  184 ->  214 ->  213 ->  281 ->  277 ->  271 ->  268 ->  272 ->  183 ->  158 ->  267 ->  110 ->  80 ->  81 ->  270 ->  50 ->  204 ->  193 ->  20 ->  113 ->  205 ->  165 ->  66 ->  166 ->  208 ->  178 ->  104 ->  177 ->  126 ->  109 ->  156 ->  137 ->  174 ->  200 ->  203 ->  202 ->  201 ->  190 ->  112 ->  146 ->  63 ->  61 ->  140 ->  64 ->  139 ->  102 ->  22 ->  23 ->  5 ->  161 ->  141 ->  72 ->  173 ->  145 ->  100 ->  95 ->  96 ->  97 ->  73 ->  98 ->  99 ->  182 ->  181 ->  25 ->  1 ->  258 ->  167 ->  130 ->  68 ->  129 ->  51 ->  90 ->  153 ->  150 ->  152 ->  89 ->  120 ->  169 ->  103 ->  149 ->  4 ->  21 ->  159 ->  160 ->  24 ->  48 ->  162 ->  151 ->  65 ->  74 ->  75 ->  59 ->  261 ->  260 ->  259 ->  15 ->  19 ->  18 ->  114 ->  93 ->  83 ->  227 ->  228 ->  116 ->  35 ->  248 ->  33 ->  34 ->  249 ->  42 ->  154 ->  43 ->  60 ->  168 ->  2 -

### Generate google maps route link

In [14]:
def generate_google_maps_route_link(coordinates):

    if len(coordinates) < 2:
        raise ValueError("At least two coordinates are required to create a route.")

    # Extract start, end, and waypoints
    start = coordinates[0]
    end = coordinates[-1]
    waypoints = coordinates[1:-1]  # All intermediate points

    # Construct the URL
    base_url = "https://www.google.com/maps/dir/?api=1"
    origin = f"{start[0]},{start[1]}"
    destination = f"{end[0]},{end[1]}"
    waypoints_str = "|".join(f"{lat},{lng}" for lat, lng in waypoints)

    # Combine into the final URL
    url = f"{base_url}&origin={origin}&destination={destination}&waypoints={waypoints_str}"
    return url



# Generate and print the link
print(generate_google_maps_route_link(ordered_routes[0][-10:]))

https://www.google.com/maps/dir/?api=1&origin=47.15381491025942,7.2828393835368965&destination=47.125835,7.2596615&waypoints=47.146832712923384,7.28281691406339|47.14357569170889,7.2789539915970565|47.13830766710951,7.276909918665245|47.133200953150144,7.268891604522404|47.130278910504046,7.269365645069798|47.13139841515758,7.26734040476859|47.13084868777095,7.266170368022973|47.12885910098569,7.264880214256999


### Folium route output
To output our calculated routes we use folium to portray the routes and their stops on a map.
The map gets saved as a html file which also could be saved and sent as a pdf.

In [11]:
import folium
import requests
 
# Create a Folium map centered at the first coordinate
start_lat, start_lng = ordered_routes[0][0]
m = folium.Map(location=[start_lat, start_lng], zoom_start=13)
 

# Add the route as a polyline to the map
for i in range(len(ordered_routes[0]) - 1):
    folium.PolyLine(
        [ordered_routes[0][i], ordered_routes[0][i + 1]],  # Connect each waypoint to the next
        color="blue",
        weight=2,
        opacity=0.7
    ).add_to(m)
 
for i in range(len(ordered_routes[1]) - 1):
    folium.PolyLine(
        [ordered_routes[1][i], ordered_routes[1][i + 1]],  # Connect each waypoint to the next
        color="red",
        weight=2,
        opacity=0.7
    ).add_to(m)
 
 
# Add a marker for the start point
folium.Marker(
    location=ordered_routes[0][0],
    popup="Start/Ende",
    icon=folium.Icon(color="green", icon="play"),
).add_to(m)
 
# Add markers for intermediate waypoints
for i, point in enumerate(ordered_routes[0][1:-1], start=1):
     folium.Marker(
         location=point,
         popup=f"Waypoint {i}",  # Display index of the waypoint
         icon=folium.Icon(color="blue", icon="map-marker"),
     ).add_to(m)
 
# Add markers for intermediate waypoints
for i, point in enumerate(ordered_routes[1][1:-1], start=1):
     folium.Marker(
         location=point,
         popup=f"Waypoint {i}",  # Display index of the waypoint
         icon=folium.Icon(color="red", icon="map-marker"),
     ).add_to(m)
 
# Save the map to an HTML file or display it in a Jupyter notebook
m.save("route_map.html")
m